# Visual P300 Notebook

Referencing [NeurotechX EEG notebook](https://neurotechx.github.io/EEG-ExPy/experiments/vp300.html).

## P300 Load and visualize data

We employ our own `preprocess.py` file to visualize data components. We also use `classify-#.ipynb`.

In [13]:
import os
from matplotlib import pyplot as plt

from collections import OrderedDict # to organize various models

import warnings
warnings.filterwarnings("ignore")

# MNE-python
from mne import Epochs, find_events

# EEG-notebooks implementations
    # implemented: load_data, _get_recording_dir
    # todo: load_subjects
from pathlib import Path
from os import path, makedirs
from datetime import datetime

In [14]:
DATA_DIR = path.join(path.expanduser("~"), "code", "eeg", "speakEEG", "data_master")

def my_recording_file(
    device_name: str, # 'neurosity'
    experiment: str, # 'visual-p300'
    session_id: int = -1, # unprovided session_id will be automatically assigned
    data_dir: str = DATA_DIR
) -> Path:

    # session id assignment
    if session_id == -1:
        session_id = 1
        while path.exists(Path(data_dir) / device_name / experiment / f"{datetime.today().strftime('%Y_%m_%d')}_session{str(session_id).zfill(2)}"):
            session_id += 1

    session_id = str(session_id).zfill(2)
    session_id = f"{datetime.today().strftime('%Y_%m_%d')}_session{session_id}"
    recording_dir = Path(data_dir) / device_name / experiment / session_id  # e.g. 'neurosity/visual-p300/2021-09-01_session01'

    if not path.exists(recording_dir):
        makedirs(recording_dir)
    
    return recording_dir

my_recording_file('neurosity', 'visual-p300')#, 1)

PosixPath('/Users/rishabh/code/eeg/speakEEG/data_master/neurosity/visual-p300/2024_10_21_session01')

In [15]:
import numpy as np
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC

import moabb
from moabb.datasets import BNCI2014_001, utils
from moabb.evaluations import CrossSessionEvaluation
from moabb.paradigms import LeftRightImagery
from moabb.pipelines.features import LogVariance

moabb.set_log_level("info")

pipelines = {}
pipelines["AM+LDA"] = make_pipeline(LogVariance(), LDA())
parameters = {"C": np.logspace(-2, 2, 10)}
clf = GridSearchCV(SVC(kernel="linear"), parameters)
pipe = make_pipeline(LogVariance(), clf)

pipelines["AM+SVM"] = pipe

In [16]:
dataset = BNCI2014_001()
dataset.subject_list = dataset.subject_list[:2]
datasets = [dataset]

datasets

In [17]:
fmin = 8
fmax = 35
paradigm = LeftRightImagery(fmin=fmin, fmax=fmax)

In [19]:
evaluation = CrossSessionEvaluation(
    paradigm=paradigm, datasets=datasets, suffix="examples", overwrite=False
)
results = evaluation.process(pipelines)

2024-10-21 06:13:46,568 INFO MainThread moabb.evaluations.base Processing dataset: BNCI2014-001
BNCI2014-001-CrossSession:   0%|          | 0/2 [00:00<?, ?it/s]Downloading data from 'http://bnci-horizon-2020.eu/database/data-sets/001-2014/A01T.mat' to file '/Users/rishabh/code/eeg/speakEEG/data/MNE-bnci-data/database/data-sets/001-2014/A01T.mat'.
100%|█████████████████████████████████████| 42.8M/42.8M [00:00<00:00, 16.0GB/s]
SHA256 hash of downloaded file: 054f02e70cf9c4ada1517e9b9864f45407939c1062c6793516585c6f511d0325
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
100%|█████████████████████████████████████| 43.8M/43.8M [00:00<00:00, 12.1GB/s]
SHA256 hash of downloaded file: 53d415f39c3d7b0c88b894d7b08d99bcdfe855ede63831d3691af1a45607fb62
Use this value as the 'known_hash' argument of 'pooch.retrieve' to ensure that the file hasn't changed if it is downloaded again in the future.
2024-10-2

In [20]:
print(results.head()) # pd dataframe

      score      time  samples subject session  channels  n_sessions  \
0  0.797068  0.283806    144.0       1  0train        22           2   
1  0.773920  0.289637    144.0       1   1test        22           2   
2  0.550733  0.429853    144.0       2  0train        22           2   
3  0.471451  0.305453    144.0       2   1test        22           2   
4  0.786458  0.156156    144.0       1  0train        22           2   

        dataset pipeline  
0  BNCI2014-001   AM+SVM  
1  BNCI2014-001   AM+SVM  
2  BNCI2014-001   AM+SVM  
3  BNCI2014-001   AM+SVM  
4  BNCI2014-001   AM+LDA  
